# Results preview available on Google Colab

Link: [Here](https://colab.research.google.com/drive/1zhXSrMmOtN3eSgCZfel-K9mhrH9153zA?usp=sharing)

In [1]:
import numpy as np
import pandas as pd
import re
from plotly.offline import plot
import plotly.graph_objs as go

In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mape_results(results):

  for x in range(0, len(results)):

    vl_true = list(results['True'])
    true_split = re.sub("['\[\]]", "", vl_true[x]).split(', ')
    vl_true = np.array([float(x) for x in true_split])

    vl_pred = list(results['Pred'])
    pred_split = re.sub("['\[\]]", "", vl_pred[x]).split(', ')
    vl_pred = np.array([float(x) for x in pred_split])

    mape = mean_absolute_percentage_error(vl_true, vl_pred)

    results.iloc[x, 3] = float(round(mape, 3))

  return results

In [3]:
def best_results(results, step):

    all_true = list(results['True'])
    true_all = re.sub("['\[\]]", "", all_true[0]).split(', ')
    all_true = np.array([float(x) for x in true_all])
    
    print("\nMelhores Resultados por Modelo - Horizonte de previsão: H = ", step)
    
    true = results.loc[(results['step'] == step) & (results['Representation'] == "ts_mul")]
    true = true.loc[true['Mape'] == true['Mape'].min(), ['True']]
    true_split = str(true.values)
    true_split = re.sub("['\[\]]", "", true_split).split(', ')
    vl_true = np.array([float(x) for x in true_split])
    #print(len(vl_true))
    
    #Menor valor de Mape - Previsão de Series Temporais Multivariada
    ts = results.loc[(results['step'] == step) & (results['Representation'] == 'ts_mul')]
    ts = ts.loc[ts['Mape'] == ts['Mape'].min()]
    print("Modelo ", ts['Model'].values, " - Mape:", ts['Mape'].values)
    ts_split = str(ts['Pred'].values)
    ts_split = re.sub("['\[\]]", "", ts_split).split(', ')
    ts_res = np.array([float(x) for x in ts_split])
    #print(len(ts_res))
    
    #Menor valor de Mape - Previsão de Series Temporais Multivariada com a Bag-of-Words 
    tsbow = results.loc[(results['step'] == step) & (results['Representation'] == 'tsed')]
    tsbow = tsbow.loc[tsbow['Mape'] == tsbow['Mape'].min()]
    print("Modelo ", tsbow['Model'].values, " - Mape:", tsbow['Mape'].values)
    tsb_split = str(tsbow['Pred'].values)
    tsb_split = re.sub("['\[\]]", "", tsb_split).split(', ')
    tsbow_res = np.array([float(x) for x in tsb_split])
    #print(len(tsbow_res))
    
    #Menor valor de Mape - Previsão com a Bag-of-Words 
    bow = results.loc[(results['step'] == step) & (results['Representation'] == 'dst')]
    bow = bow.loc[bow['Mape'] == bow['Mape'].min()]
    print("Modelo ", bow['Model'].values, " - Mape:", bow['Mape'].values)
    bow_split = str(bow['Pred'].values)
    bow_split = re.sub("['\[\]]", "", bow_split).split(', ')
    bow_res = np.array([float(x) for x in bow_split])
    #print(len(tsbow_res))
    
    return all_true, vl_true, ts_res, tsbow_res, bow_res

#Verifica os dias que os MAPEs foram iguais a zero e retorna os índices.
def get_points(res_true, ts_pred, tsb_pred, per):

  inds_tsb, inds_ts  = [], []
  best_ts, best_tsb = [], []
  quant_ts, quant_tsb, quant_eq  = 0, 0, 0

  for x in range(1, len(res_true)):

    mp_ts = mean_absolute_percentage_error(res_true[x], ts_pred[x])
    mp_tsb = mean_absolute_percentage_error(res_true[x], tsb_pred[x])

    diff = abs(mp_ts - mp_tsb)

    if (mp_tsb < 0.1) and (mp_tsb < mp_ts) and (diff > per) :
      inds_tsb.append(x)
      
    if (mp_ts < 0.1) and (mp_ts < mp_tsb) and (diff > per):
      inds_ts.append(x)

    if (mp_ts < 0.1) and (mp_ts < mp_tsb):
      best_ts.append(x)

    if (mp_tsb < 0.1) and (mp_tsb < mp_ts) :
      best_tsb.append(x)

    if mp_ts == mp_tsb:
      quant_eq += 1
    elif mp_tsb < mp_ts:
      quant_tsb += 1
    else:
      quant_ts += 1

  return inds_ts, inds_tsb, best_ts, best_tsb, quant_ts, quant_tsb, quant_eq

def print_results(results, date, step):
    
    _, vls_true, vls_ts, vls_tsed, vls_dst = best_results(results, step)
    _, _, bc_ts, bc_tsb, _, _, _ = get_points(vls_true, vls_ts, vls_tsed, 1)
            
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=date, y=vls_true, mode='lines', name='True', line=dict(color='#006400', width=1)))
    fig.add_trace(go.Scatter(x=date, y=vls_ts, mode='lines', name='Pred. TS', line=dict(color='#FF0000', width=1)))
    fig.add_trace(go.Scatter(x=date, y=vls_tsed, mode='lines', name='Pred. TSED', line=dict(color='#0000FF', width=1)))
    fig.add_trace(go.Scatter(x=date, y=vls_dst, mode='lines', name='Pred. DST', line=dict(color='#8B4513', width=1)))
    
    for pt1 in bc_ts:
        fig.add_trace(go.Scatter(x=date[pt1:pt1+1], y=vls_ts[pt1:pt1+1], mode='markers', name='TS: MAPE = 0', marker=dict(size=[9]), marker_color='rgba(255, 0, 0, .99)' ))

    for pt2 in bc_tsb:
        fig.add_trace(go.Scatter(x=date[pt2:pt2+1], y=vls_tsed[pt2:pt2+1], mode='markers', name='TSED: MAPE = 0', marker=dict(size=[9]), marker_color='rgba(0, 0, 255, .99)'))

    #fig1.add_trace(go.Scatter(x=date_corn, y=corn_bow, mode='lines', name='Value Pred BoW'))
    fig.update_layout( title="Daily values", xaxis_title="Date", yaxis_title="Dollar (US$)")
    fig.show()

    

In [17]:
def main(cmdt, stp):
    
  if cmdt == 'corn':
    #Values underline in article - CORN
    print ("\nBEST CORN RESULTS")
    results_corn = pd.read_csv('corn_res.csv', sep='\t')
    results_corn = mape_results(results_corn)

    table3_c = pd.DataFrame(index=['TS', 'TSED', 'TS=TSED', 'TS(MAPE 0%)', 'TSED(MAPE 0%)'])
    
    for s in range(1, 6):
      aux = []
      _, vls_true, vls_ts, vls_tsed, vls_dst = best_results(results_corn, s)
      ind_st, ind_stb, bc_ts, bc_tsb, qc_ts, qc_tsb, qc_eq = get_points(vls_true, vls_ts, vls_tsed, 1)

      aux = [qc_ts, qc_tsb, qc_eq, len(bc_ts), len(bc_tsb)]
      hd = "h = "+str(s)
      table3_c[hd] = aux

    print("\n\n Table 3 - Comparison of the performance of representations in number of fore-casts. \n\n" , table3_c)

    corn_st = pd.read_csv('corn.csv', sep=',')
    corn_st['Date'] = pd.to_datetime(corn_st['Date'])
    corn_st.dropna(subset=['Close', 'Open', 'High', 'Low'], inplace=True)
    corn_st.drop(columns=['Volume'], axis=1, inplace=True)
    corn_st.sort_values(by=['Date'], inplace=True)

    #Quantidade de dias para o conjunto de teste
    tam_corn = int(len(corn_st) * 0.3)
    date_corn = corn_st.iloc[tam_corn:-1, 4]

    print("\n\n--Figure shows the graph of the true and forecasted values of commodities with forecasting horizon h = ", stp)
      
    print_results(results_corn, date_corn, stp)

    #------------------------------------------------------------------------

  if cmdt == 'soybean':
    print ("\nBEST SOYBEAN RESULTS")
    results_soybean = pd.read_csv('soybean_res.csv', sep='\t')
    results_soybean = mape_results(results_soybean)
    
    table3_s = pd.DataFrame(index=['TS', 'TSED', 'TS=TSED', 'TS(MAPE 0%)', 'TSED(MAPE 0%)'])

    for s in range(1, 6):
      aux = []
      _, vls_true, vls_ts, vls_tsed, vls_dst = best_results(results_soybean, s)
      ind_st, ind_stb, bs_ts, bs_tsb, qs_ts, qs_tsb, qs_eq = get_points(vls_true, vls_ts, vls_tsed, 1)

      aux = [qs_ts, qs_tsb, qs_eq, len(bs_ts), len(bs_tsb)]
      hd = "h = "+str(s)
      table3_s[hd] = aux

    print("\n\n Table 3 - Comparison of the performance of representations in number of fore-casts. \n\n" , table3_s)

    soy_st = pd.read_csv('soybean.csv', sep=',')
    soy_st['Date'] = pd.to_datetime(soy_st['Date'])
    soy_st.dropna(subset=['Close', 'Open', 'High', 'Low'], inplace=True)
    soy_st.drop(columns=['Volume'], axis=1, inplace=True)
    soy_st.sort_values("Date", ascending=True, inplace=True)

    #Quantidade de dias para o conjunto de teste
    tam_soy = int(len(soy_st) * 0.3)
    date_soy = soy_st.iloc[tam_soy:-1, 4]

    print("\n\n--Figure shows the graph of the true and forecasted values of commodities with forecasting horizon h = ", stp)
    print_results(results_soybean, date_soy, stp)


In [18]:
#Table 2 - Best corn results - Values underline
# Figure 3 - Predicted daily value for corn and soybeans with horizon (h=1).
h = 1
main('corn', h)


BEST CORN RESULTS

Melhores Resultados por Modelo - Horizonte de previsão: H =  1
Modelo  ['SVR_RBF']  - Mape: [1.145]
Modelo  ['RandomForrest4']  - Mape: [1.168]
Modelo  ['SVR_RBF']  - Mape: [6.056]

Melhores Resultados por Modelo - Horizonte de previsão: H =  2
Modelo  ['SVR_RBF']  - Mape: [1.566]
Modelo  ['RandomForrest4']  - Mape: [1.593]
Modelo  ['SVR_RBF']  - Mape: [6.036]

Melhores Resultados por Modelo - Horizonte de previsão: H =  3
Modelo  ['SVR_RBF']  - Mape: [1.888]
Modelo  ['RandomForrest4']  - Mape: [1.929]
Modelo  ['SVR_RBF']  - Mape: [6.015]

Melhores Resultados por Modelo - Horizonte de previsão: H =  4
Modelo  ['SVR_RBF']  - Mape: [2.168]
Modelo  ['RandomForrest4']  - Mape: [2.215]
Modelo  ['SVR_RBF']  - Mape: [5.993]

Melhores Resultados por Modelo - Horizonte de previsão: H =  5
Modelo  ['SVR_RBF']  - Mape: [2.407]
Modelo  ['SVR_RBF']  - Mape: [2.45]
Modelo  ['SVR_RBF']  - Mape: [5.985]


 Table 3 - Comparison of the performance of representations in number of fore

In [19]:
#Table 2 - Best soybean results - Values underline
# Figure 3 - Predicted daily value for soybeans with horizon (h=1).

h = 1
main('soybean', h)


BEST SOYBEAN RESULTS

Melhores Resultados por Modelo - Horizonte de previsão: H =  1
Modelo  ['HGradBoostingRreg']  - Mape: [0.982]
Modelo  ['HGradBoostingRreg']  - Mape: [0.997]
Modelo  ['SVR_RBF']  - Mape: [7.611]

Melhores Resultados por Modelo - Horizonte de previsão: H =  2
Modelo  ['BaggingReg']  - Mape: [1.369]
Modelo  ['SVR_RBF']  - Mape: [1.352]
Modelo  ['SVR_RBF']  - Mape: [7.56]

Melhores Resultados por Modelo - Horizonte de previsão: H =  3
Modelo  ['BaggingReg']  - Mape: [1.659]
Modelo  ['BaggingReg']  - Mape: [1.646]
Modelo  ['SVR_RBF']  - Mape: [7.568]

Melhores Resultados por Modelo - Horizonte de previsão: H =  4
Modelo  ['BaggingReg']  - Mape: [1.906]
Modelo  ['BaggingReg']  - Mape: [1.886]
Modelo  ['SVR_RBF']  - Mape: [7.528]

Melhores Resultados por Modelo - Horizonte de previsão: H =  5
Modelo  ['BaggingReg']  - Mape: [2.104]
Modelo  ['BaggingReg']  - Mape: [2.072]
Modelo  ['SVR_RBF']  - Mape: [7.506]


 Table 3 - Comparison of the performance of representations i